In [283]:
#데이터 핸들링 라이브러리
import numpy as np
import pandas as pd

## 컬럼명 | 오리지날 =>변경
- SUM_YRMTHDAT =>집계일
   - 집계일자(YYYYMMDD, Y : 년 M : 달 D : 일)
- SUM_HR=>집계시
   - 집계시(hh, h : 시)
- VDS_CD=>VDS_ID
   - VDS(차량 감지 시스템) 장비의 고유 ID
- COCT_CD=>차로_ID
    - 차로 유형 구분 코드
- TRFFCVLM=>단위시간당_자동차_보행자수
    - 단위 시간 당 일정 지점을 통과하는 자동차, 자전거 보행자 등의 수
- OCCPNCY=>차량점거율
    - 도로별 차량 점유율
- SPD_AVG =>평균속도
    - 평균 속도
- NMLT_CSCNT => 정상_데이터수
    - 오류가 없는 정상인 데이터의 총 건수
- REVISN_CSCNT => 보정_데이터수
    - 보정 작업이 진행되었던 데이터의 총 건수
- LAST_CHANGE_DAYHMINSEC: 최종변경시각
    -Last_Chanage_Hour:	 최종변경시각

In [284]:
## 해당 경로에 있는 .csv 파일명 리스트 가져오기

import os
import time
t_0= time.time()

path = './VDS_데이터/'
file_list = os.listdir(path)
file_list_py = [file for file in file_list if file.endswith('.csv')] ## 파일명 끝이 .csv인 경우

## csv 파일들을 DataFrame으로 불러와서 concat

VDS_df = pd.DataFrame()
for i in file_list_py:
    data = pd.read_csv(path + i)
    VDS_df = pd.concat([VDS_df,data])

VDS_df = VDS_df.reset_index(drop = True)
print(f'데이터 불러오기 및 통합에 걸린 시간은 {str(time.time()-t_0)} 입니다')
VDS_df.info()

데이터 불러오기 및 통합에 걸린 시간은 0.12446427345275879 입니다
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97232 entries, 0 to 97231
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   SUM_YRMTH               24364 non-null  float64
 1   VDS_CD                  24364 non-null  object 
 2   COCT_CD                 24364 non-null  float64
 3   TRFFCVLM                24364 non-null  float64
 4   OCCPNCY                 24364 non-null  float64
 5   SPD_AVG                 24364 non-null  float64
 6   NMLT_CSCNT              24364 non-null  float64
 7   REVISN_CSCNT            24364 non-null  float64
 8   LAST_CHANGE_DAYHMINSEC  24364 non-null  float64
 9   AGGR_YM                 72868 non-null  float64
 10  VDS_ID                  72868 non-null  object 
 11  ROAD_KIND_TP_SE_CD      72868 non-null  float64
 12  VMTC                    72868 non-null  float64
 13  SHARE                   72868 non-null  float

In [285]:
#현재 VSD_DF는 2021.10월을 경계로 컬럼명이 변경되어 옆으로 쌓이는 현상이 발생했다
#명칭은 변경되었지만 새로운 데이터 컬럼이 추가되거나 데이터 순서가 변경된 것은 아니므로
VDS_df.columns

#현재 컬럼명 데이터
now_col_name=['SUM_YRMTH','VDS_CD','COCT_CD','TRFFCVLM','OCCPNCY','SPD_AVG','NMLT_CSCNT','REVISN_CSCNT',
              'LAST_CHANGE_DAYHMINSEC']
now_name_df=VDS_df[now_col_name]
now_name_df.dropna(inplace=True)

#과거 컬럼명 데이터
old_col_name=['AGGR_YM','VDS_ID','ROAD_KIND_TP_SE_CD','VMTC','SHARE','AVRG_VE','NORMAL_CNT','REVISION_CNT',
              'LAST_CHANGE_TIME']
old_name_df= VDS_df[old_col_name]
old_name_df.dropna(inplace=True)
old_name_df.columns= now_col_name

VDS_df= pd.concat([old_name_df,now_name_df]).sort_values(by='SUM_YRMTH')

/var/folders/m9/0vsmjxhd7q30k5mzw0bk58hh0000gn/T/ipykernel_5892/4043482667.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  now_name_df.dropna(inplace=True)
/var/folders/m9/0vsmjxhd7q30k5mzw0bk58hh0000gn/T/ipykernel_5892/4043482667.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  old_name_df.dropna(inplace=True)


In [286]:
### 새로운 컬럼명 지정해주기
new_col_name=['집계일','VDS_ID','차로_ID','단위시간당_자동차와보행자_수','차량점거율','평균속도','정상_데이터수',
              '보정_데이터수','최종변경시각']
VDS_df.columns= new_col_name

#재배열한 데이터를 다시 인덱싱하기
VDS_df.reset_index(inplace=True)
VDS_df.drop(['index'],axis=1,inplace=True)
VDS_df

,집계일,VDS_ID,차로_ID,단위시간당_자동차와보행자_수,차량점거율,평균속도,정상_데이터수,보정_데이터수,최종변경시각
0,202102.0,0010VDE00100,1.0,1026214.0,1.90,84.53,79470.0,587.0,2.021030e+13
1,202102.0,0520VDE03700,1.0,592743.0,2.92,108.40,79781.0,0.0,2.021030e+13
2,202102.0,0520VDE03600,1.0,-1.0,-1.00,-1.00,79781.0,0.0,2.021030e+13
3,202102.0,0520VDE03500,1.0,604691.0,2.92,109.01,79781.0,0.0,2.021030e+13
4,202102.0,0520VDE03400,1.0,605059.0,3.10,101.38,79781.0,0.0,2.021030e+13
...,...,...,...,...,...,...,...,...,...
97227,202202.0,0520VDS01100,1.0,706038.0,5.29,97.39,80513.0,0.0,2.022030e+13
97228,202202.0,0520VDS01200,1.0,704518.0,4.74,94.89,80513.0,0.0,2.022030e+13
97229,202202.0,0520VDS01300,1.0,709232.0,4.56,97.93,80513.0,0.0,2.022030e+13
97230,202202.0,0520VDE06200,1.0,267271.0,1.48,110.40,80513.0,0.0,2.022030e+13


In [287]:
# VDS 데이터 셋의 정보 확인하기
print(VDS_df.info())
df_stats= VDS_df.describe().T
df_stats

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97232 entries, 0 to 97231
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   집계일              97232 non-null  float64
 1   VDS_ID           97232 non-null  object 
 2   차로_ID            97232 non-null  float64
 3   단위시간당_자동차와보행자_수  97232 non-null  float64
 4   차량점거율            97232 non-null  float64
 5   평균속도             97232 non-null  float64
 6   정상_데이터수          97232 non-null  float64
 7   보정_데이터수          97232 non-null  float64
 8   최종변경시각           97232 non-null  float64
dtypes: float64(8), object(1)
memory usage: 6.7+ MB
None


,count,mean,std,min,25%,50%,75%,max
집계일,97232.0,2.021225e+05,3.550944e+01,2.021020e+05,2.021050e+05,2.021080e+05,2.021120e+05,2.022020e+05
차로_ID,97232.0,1.051115e+00,2.202331e-01,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,2.000000e+00
단위시간당_자동차와보행자_수,97232.0,5.846947e+05,5.456674e+05,-1.000000e+00,2.101520e+05,4.791600e+05,7.776768e+05,7.106895e+06
차량점거율,97232.0,3.440718e+00,3.367836e+00,-1.000000e+00,1.850000e+00,3.040000e+00,4.520000e+00,9.567000e+01
평균속도,97232.0,8.545110e+01,3.290416e+01,-1.000000e+00,8.863000e+01,9.658000e+01,1.024300e+02,1.655000e+02
정상_데이터수,97232.0,7.235969e+04,2.812999e+04,0.000000e+00,7.700600e+04,8.575400e+04,8.860000e+04,1.386540e+05
보정_데이터수,97232.0,1.678521e+03,5.707797e+03,0.000000e+00,0.000000e+00,4.900000e+01,6.300000e+02,5.398400e+04
최종변경시각,97232.0,2.021309e+13,4.119915e+09,2.021030e+13,2.021060e+13,2.021090e+13,2.022010e+13,2.022030e+13


In [288]:
#결측치 확인
VDS_df.isnull().sum()

집계일                0
VDS_ID             0
차로_ID              0
단위시간당_자동차와보행자_수    0
차량점거율              0
평균속도               0
정상_데이터수            0
보정_데이터수            0
최종변경시각             0
dtype: int64

In [289]:
#상대적으로 중요도가 적어보이는 컬럼 삭제
VDS_df.drop(['정상_데이터수','보정_데이터수','최종변경시각'],axis=1,inplace=True)


In [290]:
VDS_df.groupby('차로_ID').sum()

,집계일,단위시간당_자동차와보행자_수,차량점거율,평균속도
차로_ID,,,,
1.0,1.864822e+10,5.656105e+10,312481.28,7894693.49
2.0,1.004550e+09,2.899833e+08,22066.61,413888.31


In [291]:
#데이터 정의서 불러오기
lable_def= pd.read_csv('./VDS_고속도로_데이터정의서 00.11.27.csv',header=1)

In [292]:
lable_def

,VDS_ID,GRS80X좌표값,GRS80Y좌표값
0,0010VDE00100,390524.290125,296459.942624
1,0010VDE00200,390589.576852,296535.747996
2,0010VDE00300,390654.863578,296611.553368
3,0010VDE00400,390720.150305,296687.358740
4,0010VDE00500,390785.437032,296763.164111
...,...,...,...
45949,NaN,402773.478106,300965.275385
45950,NaN,402794.523435,301062.975344
45951,NaN,402812.316043,301161.379729
45952,NaN,402829.419219,301259.899480


In [293]:
# VDS_ID를 key값으로 하여 VDS_df와 label_df를 결합시킨다.
VDS_df=pd.merge(VDS_df,lable_def,how='inner',on='VDS_ID')
VDS_df

,집계일,VDS_ID,차로_ID,단위시간당_자동차와보행자_수,차량점거율,평균속도,GRS80X좌표값,GRS80Y좌표값
0,202102.0,0010VDE00100,1.0,1026214.0,1.90,84.53,390524.290125,296459.942624
1,202103.0,0010VDE00100,1.0,1131073.0,1.87,84.32,390524.290125,296459.942624
2,202104.0,0010VDE00100,1.0,1116602.0,1.91,84.37,390524.290125,296459.942624
3,202105.0,0010VDE00100,1.0,1175789.0,1.93,83.98,390524.290125,296459.942624
4,202106.0,0010VDE00100,1.0,1125649.0,1.91,83.87,390524.290125,296459.942624
...,...,...,...,...,...,...,...,...
94020,202109.0,0170VDE00025,1.0,1192235.0,11.98,84.65,283778.161955,402439.124189
94021,202110.0,0170VDE00025,1.0,1271618.0,13.12,86.16,283778.161955,402439.124189
94022,202112.0,0170VDE00025,1.0,1151903.0,11.10,86.65,283778.161955,402439.124189
94023,202201.0,0170VDE00025,1.0,1109754.0,10.14,88.44,283778.161955,402439.124189


In [294]:
from pyproj import CRS, Transformer
import pandas as pd

def project_array(coord, before_coord, after_coord):
    #coord가 DataFrame 타입으로 들어오는 경우 넘파이로 변환시켜서 데이터를 처리시키기
    #그냥 numpy인 경우엔 상관없다
    coord= coord.to_numpy()

    transformer= Transformer.from_crs(before_coord,after_coord)
    after_x,after_y= transformer.transform(coord[:,0],coord[:,1])
    after_df= pd.DataFrame(after_x,after_y)
    after_df.reset_index(inplace=True)
    after_df.columns=['경도','위도']
    after_df= after_df[['위도','경도']]
    return after_df


# EPS: 4166,4326: 전세계를 나타낼 때 쓰이는 표준적인 좌표계
# GRS80,EPSG:5185,5186 현재 국토지리정보원 표준: 2002년 이후에 국토지리정보원 지형도에서 사용중인 좌표계
GRS80X_coords=VDS_df[['GRS80X좌표값','GRS80Y좌표값']]
coord_df= project_array(GRS80X_coords,'EPSG:5180','EPSG:4326')
VDS_df[['GRS80X좌표값','GRS80Y좌표값']]=coord_df
VDS_df.rename({'GRS80X좌표값':'위도','GRS80Y좌표값':'경도'},axis=1,inplace=True)
VDS_df.drop(['VDS_ID','차로_ID'],axis=1,inplace=True)

In [295]:
#한국도로공사 GRS80 위치정보를 담은 데이터셋 불러오기
# GRS80_df=pd.read_csv('/Users/wooseongkyun/코드_아카이브/프로젝트2_데이터전처리/ETC_S0_07_04_557329 2.csv',encoding='euc-kr')
# VDS_df=pd.merge(VDS_df,GRS80_df,how='inner',on=['GRS80X좌표값','GRS80Y좌표값'])
# VDS_df.drop(['GRS80X좌표값','GRS80Y좌표값'],axis=1,inplace=True)
# VDS_df.rename({'X좌표값':'위도','Y좌표값':'경도'},axis=1,inplace=True)

In [296]:
VDS_df.head(20)



,집계일,단위시간당_자동차와보행자_수,차량점거율,평균속도,위도,경도
0,202102.0,1026214.0,1.90,84.53,37.008670,126.083783
1,202103.0,1131073.0,1.87,84.32,37.008670,126.083783
2,202104.0,1116602.0,1.91,84.37,37.008670,126.083783
3,202105.0,1175789.0,1.93,83.98,37.008670,126.083783
4,202106.0,1125649.0,1.91,83.87,37.008670,126.083783
5,202107.0,1140172.0,1.88,82.82,37.008670,126.083783
6,202108.0,1111045.0,1.85,82.20,37.008670,126.083783
7,202109.0,1134199.0,1.93,82.81,37.008670,126.083783
8,202110.0,1187299.0,1.95,83.15,37.008670,126.083783
9,202112.0,1127113.0,1.87,83.67,37.008670,126.083783


In [297]:
#맵제작에 필요한 모듈
import folium
from folium.plugins import MarkerCluster
from folium import plugins

#좌표값
latitude= np.array(VDS_df['위도']).tolist();
longitude=np.array(VDS_df['경도']).tolist();

#지도 만들기
my_location=[37.55,126.98]
VDS_map = folium.Map(location=my_location, zoom_start=12.0,
                    tiles=None, control_scale=True)
folium.TileLayer('openstreetmap', name='구분').add_to(VDS_map);

#마커 클러스터
marker_cluster = MarkerCluster().add_to(VDS_map)

for i in range(len(latitude)):
    folium.Marker(
        location = [latitude[i],longitude[i]],
        icon=folium.Icon(color='cadetblue',icon='ok')
    ).add_to(marker_cluster)

#지도저장
VDS_map.save('./VDS_시각화2.html');


In [298]:
longitude

[126.08378255683847,
 126.08378255683847,
 126.08378255683847,
 126.08378255683847,
 126.08378255683847,
 126.08378255683847,
 126.08378255683847,
 126.08378255683847,
 126.08378255683847,
 126.08378255683847,
 126.08378255683847,
 126.08378255683847,
 125.61604110428748,
 125.61604110428748,
 125.61604110428748,
 125.61604110428748,
 125.61604110428748,
 125.61604110428748,
 125.61604110428748,
 125.61604110428748,
 125.61604110428748,
 125.61604110428748,
 125.61604110428748,
 125.61604110428748,
 125.61562055211606,
 125.61562055211606,
 125.61562055211606,
 125.61562055211606,
 125.61562055211606,
 125.61562055211606,
 125.61562055211606,
 125.61562055211606,
 125.61562055211606,
 125.61562055211606,
 125.61562055211606,
 125.61562055211606,
 125.61514948513155,
 125.61514948513155,
 125.61514948513155,
 125.61514948513155,
 125.61514948513155,
 125.61514948513155,
 125.61514948513155,
 125.61514948513155,
 125.61514948513155,
 125.61514948513155,
 125.61514948513155,
 125.61514948